# 爬蟲

In [ ]:
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
import random
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

In [ ]:
option = webdriver.ChromeOptions() # 設定 chromedriver
# option.add_argument('--headless') # 無頭模式，開發完成之後再使用，可以完全背景執行，有機會變快。但無頭模式在某些網站會不能爬。
option.add_experimental_option('excludeSwitches', ['enable-automation']) # 開發者模式。可以避開某些防爬機制，有開有保佑
driver = webdriver.Chrome(options=option) # 啟動 chromedriver
driver.maximize_window() # 視窗放到最大。有些網站的版面會隨視窗大小改變，在沒有放到最大的情況下有可能定位正確但戳到其他地方，所以通常會放大

url = 'https://kktix.com/events?utf8=%E2%9C%93&search=&max_price=&min_price=&start_at=&end_at=&event_tag_ids_in=1' # 指定網址
driver.get(url) # 進入指定網址
driver.implicitly_wait(5) # 等待伺服器反應最多 5 秒，如果在時間到之前反應就提早結束等待

page_number = 1
all_events = 0

event_name = []
event_user_time = []
venue = []
event_address = []
event_link = []
img_link = []

while True:

    processed_event = 1
    events = driver.find_elements(By.CSS_SELECTOR, 'span.fake-btn.pull-right')

    #當前頁面活動未處理完時
    while processed_event <= len(events):

        #找到此頁所有活動(定位)
        events = driver.find_elements(By.CSS_SELECTOR, 'span.fake-btn.pull-right')

        #遍歷活動頁面(從未處理開始)
        for event_index in range(processed_event - 1, len(events)):
            #點選活動頁
            try:
                event = events[event_index]
                event.click()
                sleep(1 + random.random())
                print(f'Page {page_number}  / event {processed_event}')

                #若有大標題則抓取資料
                try:
                    title = driver.find_element(By.CSS_SELECTOR, 'body > div.outer-wrapper > div.content-wrapper > div > div.header > div > h1')
                    # 篩選掉特定活動名稱
                    filter_out = ["福利","香港","HongKong","Hong Kong","HONGKONG","HONG KONG","MACAU","信用卡","教育振興券","ㄉㄉ","讚美之泉","專輯預購"]
                    # 檢查變數，預設為正常活動（需爬）
                    check = True

                    for word in filter_out:
                        if word in title.text:
                            check = False
                            break

                    if check == True:
                        print(f'event: {title.text}')
                        event_name.append(title.text)


                        #定位日期
                        date = driver.find_elements('css selector', 'body > div.outer-wrapper > div.content-wrapper > div > div.event-info > ul > li:nth-child(1) > span > span:nth-child(2)')
                        for date_element in date:
                            dates = date_element.text.replace("(+0800)","")
                            print(dates)
                            event_user_time.append(dates)


                        # #處理日期格式
                        # raw_date = dates.split('(周')
                        # date = raw_date[0]
                        # weekday = '(' + raw_date[1].split(') ')[0] + ')'
                        # time = raw_date[1].split(') ')[1].split('(')[0]
                        # formatted_date = f'{date} {weekday} {time}'
                        # print(f'date: {formatted_date}')

                        #定位地點與地址
                        location = driver.find_elements('css selector', 'body > div.outer-wrapper > div.content-wrapper > div > div.event-info > ul > li:nth-child(2) > span')
                        for location_element in location:
                            raw_location = location_element.text
                            if '/' in raw_location:
                                #分開地點與地址
                                location = raw_location.split(' / ')[0]
                                address = raw_location.split(' / ')[1]
                                print(f'location: {location}')
                                print(f'address: {address}')
                                venue.append(location)
                                event_address.append(address)
                            else:
                                print(f'location: {raw_location}')
                                venue.append(raw_location)

                        #網址
                        current_url = driver.current_url
                        print(f'link: {current_url}')
                        event_link.append(current_url)

                        #圖片
                        image = driver.find_element('css selector', 'body > div.outer-wrapper > div.content-wrapper > div > div.og-banner > img')
                        image_url = image.get_attribute('src')
                        print(f'image: {image_url}')
                        print()
                        img_link.append(image_url)

                    # 活動名稱篩選沒通過就跳過不爬
                    else:
                        pass

                #若無大標題則跳過頁面
                except NoSuchElementException:
                    print('skip this page')
                    print()

                #處理完活動頁則跳回上頁，處理完活動數+1
                finally:
                    driver.back()
                    processed_event += 1
                    all_events += 1

            #元素引用失效異常(可能由於頁面加載不完全?)，需重新定位元素
            except StaleElementReferenceException:
                driver.back()
                break

    #遍歷完12個活動 進入下一頁
    page_number += 1
    driver.get(f'https://kktix.com/events?end_at=&event_tag_ids_in=1&max_price=&min_price=&page={page_number}&search=&start_at=')
    sleep(3)  # 給予頁面足夠的時間加載

    events = driver.find_elements(By.CSS_SELECTOR, 'span.fake-btn.pull-right')
    if events:
        pass
    else:
        print(f'(已完成爬蟲，總活動數: {all_events} 個)')
        break

driver.close()

In [ ]:
print(event_name)
print(event_user_time)
print(event_address)
print(venue)
print(img_link)
print(event_link)

# 整理StartTime,EndTime

In [ ]:
# 把event_user_time 存成 StartTime, EndTime（KKTIX沒有endtime所以都設為同一天）
start_time = []
end_time = []

for time in event_user_time:
    date = time.split('(')[0]
    new_date = date.replace("/","-")
    start_time.append(new_date)
    end_time.append(new_date)

print(start_time)
print(end_time)

# 整理所有資料成dataframe

In [ ]:
import pandas as pd
# 轉成dataframe

# 把各個資料list使用pandas存成dataframe
kktix_df = pd.DataFrame(list(zip(event_name,event_user_time,venue,event_address,img_link,event_link,start_time,end_time)), columns = ['EventName', 'EventTime', 'Venue', 'Address', 'ImageURL', 'PageURL', 'StartTime', 'EndTime'])

kktix_df

# 連線DB並存入df

In [ ]:
# 連線資料庫並存入DB

import psycopg2
import pandas as pd
from sqlalchemy import create_engine

# 此py檔案是本地做資料庫連線、建table、存爬蟲資料進table用的
# 僅供本地用，放上來給大家參考，主程式並沒有用到


# Postgres DB 連線
# 依照render上的參數輸入
conn = psycopg2.connect(
    dbname='',
    user='',
    password='',
    host='dpg--a.singapore-postgres.render.com',
    port='5432'
)

cursor = conn.cursor()

# Create table (if it doesn't exist)
create_table_query = '''
CREATE TABLE IF NOT EXISTS tb_kktix (
    id SERIAL PRIMARY KEY,
    EventName VARCHAR(255) NOT NULL,
    EventTime VARCHAR(255),
    Venue VARCHAR(255),
    Address VARCHAR(255),
    ImageURL VARCHAR(1000),
    PageURL VARCHAR(1000),
    StartTime TIMESTAMP,
    EndTime TIMESTAMP
);
'''
cursor.execute(create_table_query)
conn.commit()
print("Table created successfully.")


# 利用SQLAlchemy將爬蟲資料放入Postgres DB (因Postgres非內建資料庫，需要SQLAlchemy去方便管理及簡化code)

# PostgreSQL connection details
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'dpg--a.singapore-postgres.render.com'  # Replace with your endpoint
USER = ''  # Replace with your username
PASSWORD = ''  # Replace with your password
PORT = 5432  # Default PostgreSQL port
DATABASE = ''  # Replace with your database name

# Create SQLAlchemy engine
engine = create_engine(f'{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}')

# Insert the data into the PostgreSQL table
# 此假設df已經儲存爬蟲爬下來並且用pandas整理過的dataframe
kktix_df.to_sql('tb_kktix', engine, if_exists='replace', index=False)


print("Data inserted successfully.")

conn.close()